# GCN practice code

- import basic library

In [102]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

import networkx as nx
import numpy as np
from random import randint, expovariate
import matplotlib.pyplot as plt

- Generate the network

In [103]:
S_CPU_MAX = []
S_BW_MAX = []
net = nx.gnm_random_graph(n=20, m=100)

min_cpu_capacity = 1.0e10
max_cpu_capacity = 0.0
for node_id in net.nodes:
    net.nodes[node_id]['CPU'] = randint(50, 100)
    net.nodes[node_id]['LOCATION'] = randint(0, 2)
    if net.nodes[node_id]['CPU'] < min_cpu_capacity:
        min_cpu_capacity = net.nodes[node_id]['CPU']
    if net.nodes[node_id]['CPU'] > max_cpu_capacity:
        max_cpu_capacity = net.nodes[node_id]['CPU']
        
min_bandwidth_capacity = 1.0e10
max_bandwidth_capacity = 0.0
for edge_id in net.edges:
    net.edges[edge_id]['bandwidth'] = randint(50, 100)
    if net.edges[edge_id]['bandwidth'] < min_bandwidth_capacity:
        min_bandwidth_capacity = net.edges[edge_id]['bandwidth']
    if net.edges[edge_id]['bandwidth'] > max_bandwidth_capacity:
        max_bandwidth_capacity = net.edges[edge_id]['bandwidth']
        
for s_node_id, s_node_data in net.nodes(data=True):
    S_CPU_MAX.append(s_node_data['CPU'])
# S_BW_Free
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    S_BW_MAX.append(total_node_bandwidth)

In [104]:
 # S_CPU_Free
s_CPU_remaining = []
s_bandwidth_remaining = []
current_embedding = [0] * len(net.nodes)

for s_node_id, s_node_data in net.nodes(data=True):
    s_CPU_remaining.append(s_node_data['CPU'])
# S_BW_Free
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    s_bandwidth_remaining.append(total_node_bandwidth)

In [105]:
substrate_features = []
substrate_features.append(S_CPU_MAX)
substrate_features.append(S_BW_MAX)
substrate_features.append(s_CPU_remaining)
substrate_features.append(s_bandwidth_remaining)
substrate_features.append(current_embedding)

print(substrate_features)

[[85, 96, 71, 94, 66, 62, 93, 72, 83, 67, 80, 87, 96, 78, 91, 97, 53, 67, 61, 99], [964.0, 758.0, 925.0, 790.0, 528.0, 685.0, 800.0, 1047.0, 510.0, 801.0, 1003.0, 760.0, 478.0, 624.0, 851.0, 679.0, 768.0, 689.0, 755.0, 679.0], [85, 96, 71, 94, 66, 62, 93, 72, 83, 67, 80, 87, 96, 78, 91, 97, 53, 67, 61, 99], [964.0, 758.0, 925.0, 790.0, 528.0, 685.0, 800.0, 1047.0, 510.0, 801.0, 1003.0, 760.0, 478.0, 624.0, 851.0, 679.0, 768.0, 689.0, 755.0, 679.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [106]:
substrate_features = torch.tensor(substrate_features)
print(substrate_features)
print(substrate_features.shape)
substrate_features = torch.transpose(substrate_features, 0, 1)
print(substrate_features)
print(substrate_features.shape)

tensor([[  85.,   96.,   71.,   94.,   66.,   62.,   93.,   72.,   83.,   67.,
           80.,   87.,   96.,   78.,   91.,   97.,   53.,   67.,   61.,   99.],
        [ 964.,  758.,  925.,  790.,  528.,  685.,  800., 1047.,  510.,  801.,
         1003.,  760.,  478.,  624.,  851.,  679.,  768.,  689.,  755.,  679.],
        [  85.,   96.,   71.,   94.,   66.,   62.,   93.,   72.,   83.,   67.,
           80.,   87.,   96.,   78.,   91.,   97.,   53.,   67.,   61.,   99.],
        [ 964.,  758.,  925.,  790.,  528.,  685.,  800., 1047.,  510.,  801.,
         1003.,  760.,  478.,  624.,  851.,  679.,  768.,  689.,  755.,  679.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]])
torch.Size([5, 20])
tensor([[  85.,  964.,   85.,  964.,    0.],
        [  96.,  758.,   96.,  758.,    0.],
        [  71.,  925.,   71.,  925.,    0.],
        [  94.,  790.,   94.,  790.,    0.],
    

In [107]:
substrate_features = torch.reshape(substrate_features, (-1,))
print(substrate_features.shape)
print(substrate_features)

torch.Size([100])
tensor([  85.,  964.,   85.,  964.,    0.,   96.,  758.,   96.,  758.,    0.,
          71.,  925.,   71.,  925.,    0.,   94.,  790.,   94.,  790.,    0.,
          66.,  528.,   66.,  528.,    0.,   62.,  685.,   62.,  685.,    0.,
          93.,  800.,   93.,  800.,    0.,   72., 1047.,   72., 1047.,    0.,
          83.,  510.,   83.,  510.,    0.,   67.,  801.,   67.,  801.,    0.,
          80., 1003.,   80., 1003.,    0.,   87.,  760.,   87.,  760.,    0.,
          96.,  478.,   96.,  478.,    0.,   78.,  624.,   78.,  624.,    0.,
          91.,  851.,   91.,  851.,    0.,   97.,  679.,   97.,  679.,    0.,
          53.,  768.,   53.,  768.,    0.,   67.,  689.,   67.,  689.,    0.,
          61.,  755.,   61.,  755.,    0.,   99.,  679.,   99.,  679.,    0.])


In [114]:
vnr_cpu = torch.tensor([10])
vnr_bw = torch.tensor([30])
pending = torch.tensor([2])
substrate_features = torch.cat((substrate_features, vnr_cpu, vnr_bw, pending), 0)

In [115]:
print(substrate_features)

tensor([  85.,  964.,   85.,  964.,    0.,   96.,  758.,   96.,  758.,    0.,
          71.,  925.,   71.,  925.,    0.,   94.,  790.,   94.,  790.,    0.,
          66.,  528.,   66.,  528.,    0.,   62.,  685.,   62.,  685.,    0.,
          93.,  800.,   93.,  800.,    0.,   72., 1047.,   72., 1047.,    0.,
          83.,  510.,   83.,  510.,    0.,   67.,  801.,   67.,  801.,    0.,
          80., 1003.,   80., 1003.,    0.,   87.,  760.,   87.,  760.,    0.,
          96.,  478.,   96.,  478.,    0.,   78.,  624.,   78.,  624.,    0.,
          91.,  851.,   91.,  851.,    0.,   97.,  679.,   97.,  679.,    0.,
          53.,  768.,   53.,  768.,    0.,   67.,  689.,   67.,  689.,    0.,
          61.,  755.,   61.,  755.,    0.,   99.,  679.,   99.,  679.,    0.,
          10.,   30.,    2.])


- Using 'from_networkx'
    - transfer the torch_geometric

In [6]:
data = from_networkx(net)

In [7]:
print(data)

Data(CPU=[20], LOCATION=[20], bandwidth=[200], edge_index=[2, 200])


### Graph Convolution Network
- Generate the GCN class

In [89]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(in_channels=5, out_channels=4)
        self.conv2 = GCNConv(in_channels=4, out_channels=4)
        self.conv3 = GCNConv(in_channels=4, out_channels=2)
        self.classifier = Linear(2, 20)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(5, 4)
  (conv2): GCNConv(4, 4)
  (conv3): GCNConv(4, 2)
  (classifier): Linear(in_features=2, out_features=20, bias=True)
)


In [90]:
model = GCN()

out, embedding = model(substrate_features, data.edge_index)
print(f'Embedding shape: {list(embedding.shape)}')

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [91]:
print(embedding)
print(embedding.shape)

tensor([[ 0.5998, -0.7722],
        [ 0.7210, -0.8732],
        [ 0.6010, -0.7732],
        [ 0.6248, -0.7948],
        [ 0.5776, -0.7511],
        [ 0.6629, -0.8277],
        [ 0.6584, -0.8240],
        [ 0.5798, -0.7531],
        [ 0.5982, -0.7707],
        [ 0.6028, -0.7749],
        [ 0.6600, -0.8253],
        [ 0.6277, -0.7974],
        [ 0.6219, -0.7923],
        [ 0.5796, -0.7530],
        [ 0.5782, -0.7516],
        [ 0.5983, -0.7707],
        [ 0.5266, -0.6997],
        [ 0.6415, -0.8096],
        [ 0.6064, -0.7780],
        [ 0.7071, -0.8628]], grad_fn=<TanhBackward>)
torch.Size([20, 2])


In [92]:
print(out.shape)
print(out)

torch.Size([20, 20])
tensor([[ 0.3816, -1.2462,  0.5695,  0.1170, -0.6605, -0.7185, -0.7190,  0.6389,
          0.7770, -0.1771,  0.1242,  0.4228,  0.0096, -0.2680,  0.1743, -0.2620,
         -1.3410, -0.1200,  0.2875, -0.1379],
        [ 0.4609, -1.3267,  0.6344,  0.2160, -0.6658, -0.7883, -0.7665,  0.7383,
          0.8392, -0.1539,  0.1131,  0.4253,  0.0094, -0.2943,  0.1922, -0.3628,
         -1.4549, -0.1028,  0.2432, -0.1585],
        [ 0.3823, -1.2470,  0.5702,  0.1180, -0.6605, -0.7192, -0.7194,  0.6399,
          0.7776, -0.1769,  0.1240,  0.4228,  0.0096, -0.2683,  0.1744, -0.2630,
         -1.3421, -0.1199,  0.2871, -0.1381],
        [ 0.3987, -1.2638,  0.5839,  0.1382, -0.6616, -0.7334, -0.7295,  0.6604,
          0.7898, -0.1731,  0.1221,  0.4239,  0.0084, -0.2746,  0.1778, -0.2839,
         -1.3654, -0.1160,  0.2776, -0.1412],
        [ 0.3659, -1.2301,  0.5562,  0.0977, -0.6595, -0.7049, -0.7093,  0.6193,
          0.7657, -0.1802,  0.1258,  0.4215,  0.0114, -0.2615,  0.

# A3C Code
- Simple A3C code

In [93]:
from torch import nn
import torch
import numpy as np


def v_wrap(np_array, dtype=np.float32):
    if np_array.dtype != dtype:
        np_array = np_array.astype(dtype)
    return torch.from_numpy(np_array)


def set_init(layers):
    for layer in layers:
        nn.init.normal_(layer.weight, mean=0., std=0.1)
        nn.init.constant_(layer.bias, 0.)


def push_and_pull(opt, lnet, gnet, done, s_, bs, ba, br, gamma):
    if done:
        v_s_ = 0.               # terminal
    else:
        v_s_ = lnet.forward(v_wrap(s_[None, :]))[-1].data.numpy()[0, 0]

    buffer_v_target = []
    for r in br[::-1]:    # reverse buffer r
        v_s_ = r + gamma * v_s_
        buffer_v_target.append(v_s_)
    buffer_v_target.reverse()
    loss = lnet.loss_func(
        v_wrap(np.vstack(bs)),
        v_wrap(np.array(ba), dtype=np.int64) if ba[0].dtype == np.int64 else v_wrap(np.vstack(ba)),
        v_wrap(np.array(buffer_v_target)[:, None]))

    # calculate local gradients and push local parameters to global
    opt.zero_grad()
    loss.backward()
    for lp, gp in zip(lnet.parameters(), gnet.parameters()):
        gp._grad = lp.grad
    opt.step()

    # pull global parameters
    lnet.load_state_dict(gnet.state_dict())


def record(global_ep, global_ep_r, ep_r, res_queue, name):
    with global_ep.get_lock():
        global_ep.value += 1
    with global_ep_r.get_lock():
        if global_ep_r.value == 0.:
            global_ep_r.value = ep_r
        else:
            global_ep_r.value = global_ep_r.value * 0.99 + ep_r * 0.01
    res_queue.put(global_ep_r.value)
    print(
        name,
        "Ep:", global_ep.value,
        "| Ep_r: %.0f" % global_ep_r.value,
    )

In [94]:
class SharedAdam(torch.optim.Adam):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8,
                 weight_decay=0):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        # State initialization
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = torch.zeros_like(p.data)
                state['exp_avg_sq'] = torch.zeros_like(p.data)

                # share in memory
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

In [95]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import gym
import os
os.environ["OMP_NUM_THREADS"] = "1"

UPDATE_GLOBAL_ITER = 5
GAMMA = 0.9
MAX_EP = 3000

env = gym.make('CartPole-v0')
N_S = env.observation_space.shape[0]
N_A = env.action_space.n

print(N_S, N_A)

4 2


In [96]:
class Net(nn.Module):
    def __init__(self, s_dim, a_dim):
        super(Net, self).__init__()
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.pi1 = nn.Linear(s_dim, 128)
        self.pi2 = nn.Linear(128, a_dim)
        self.v1 = nn.Linear(s_dim, 128)
        self.v2 = nn.Linear(128, 1)
        set_init([self.pi1, self.pi2, self.v1, self.v2])
        self.distribution = torch.distributions.Categorical

    def forward(self, x):
        pi1 = torch.tanh(self.pi1(x))
        logits = self.pi2(pi1)
        v1 = torch.tanh(self.v1(x))
        values = self.v2(v1)
        return logits, values

    def choose_action(self, s):
        self.eval()
        logits, _ = self.forward(s)
        prob = F.softmax(logits, dim=1).data
        m = self.distribution(prob)
        return m.sample().numpy()[0]

    def loss_func(self, s, a, v_t):
        self.train()
        logits, values = self.forward(s)
        td = v_t - values
        c_loss = td.pow(2)
        
        probs = F.softmax(logits, dim=1)
        m = self.distribution(probs)
        exp_v = m.log_prob(a) * td.detach().squeeze()
        a_loss = -exp_v
        total_loss = (c_loss + a_loss).mean()
        return total_loss

In [97]:
class Worker(mp.Process):
    def __init__(self, gnet, opt, global_ep, global_ep_r, res_queue, name):
        super(Worker, self).__init__()
        self.name = 'w%02i' % name
        self.g_ep, self.g_ep_r, self.res_queue = global_ep, global_ep_r, res_queue
        self.gnet, self.opt = gnet, opt
        self.lnet = Net(N_S, N_A)           # local network
        self.env = gym.make('CartPole-v0').unwrapped

    def run(self):
        total_step = 1
        while self.g_ep.value < MAX_EP:
            s = self.env.reset()
            buffer_s, buffer_a, buffer_r = [], [], []
            ep_r = 0.
            while True:
#                 if self.name == 'w00':
#                     self.env.render()
                a = self.lnet.choose_action(v_wrap(s[None, :]))
                s_, r, done, _ = self.env.step(a)
                if done: r = -1
                ep_r += r
                buffer_a.append(a)
                buffer_s.append(s)
                buffer_r.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:  # update global and assign to local net
                    # sync
                    push_and_pull(self.opt, self.lnet, self.gnet, done, s_, buffer_s, buffer_a, buffer_r, GAMMA)
                    buffer_s, buffer_a, buffer_r = [], [], []

                    if done:  # done and print information
                        record(self.g_ep, self.g_ep_r, ep_r, self.res_queue, self.name)
                        break
                s = s_
                total_step += 1
        self.res_queue.put(None)

In [99]:
gnet = Net(N_S, N_A)        # global network
gnet.share_memory()         # share the global parameters in multiprocessing
opt = SharedAdam(gnet.parameters(), lr=1e-4, betas=(0.92, 0.999))      # global optimizer
global_ep, global_ep_r, res_queue = mp.Value('i', 0), mp.Value('d', 0.), mp.Queue()
print(mp.cpu_count())

# parallel training
workers = [Worker(gnet, opt, global_ep, global_ep_r, res_queue, i) for i in range(mp.cpu_count())]
[w.start() for w in workers]
res = []                    # record episode reward to plot
while True:
    r = res_queue.get()
    if r is not None:
        res.append(r)
    else:
        break
[w.join() for w in workers]

import matplotlib.pyplot as plt
plt.plot(res)
plt.ylabel('Moving average ep reward')
plt.xlabel('Step')
plt.show()

48


KeyboardInterrupt: 